# Digit Recognizer
## Kaggle competition
### Learn computer vision fundamentals with the famous MNIST data

### Import Data
Dataset is downloaded from https://www.kaggle.com/c/digit-recognizer/data?select=test.csv

In [65]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
train_data = np.array(train)
train_tar = train_data[:,0]
train_data = train_data[:,1:]
train_data = 0.99*train_data/255 + 0.1

imagesize = 28
imagenumb = len(train_tar)

train_target = np.zeros((len(train_tar),10))
for i in range(len(train_target)):
    train_target[i,train_tar[i]] = 1

### Implement Neural Network


In [66]:
def sigmoid(x):
    return 1/(1+np.e ** -x)
def d_sigmoid(x):
    return x * (1-x)

activation_func = sigmoid
d_activation_func = d_sigmoid

NN_architecture = [
    {"inputs":imagesize **2, "neurons":128},
    {"inputs":128,            "neurons":64},
    {"inputs":64,            "neurons":32},
    {"inputs":32,            "neurons":16},
    {"inputs":16,            "neurons":10}
]

def forward_single_layer(out_prev, w_curr, b_curr):
        return activation_func(np.dot(w_curr, out_prev)+b_curr)

class NeuralNetwork:
    def __init__(self, architecture):
        self.architecture = architecture
        self.parameters = {}
        for idx, layer in enumerate(architecture):
            layer_idx = idx+1
            layer_input = layer["inputs"]
            layer_output = layer["neurons"]
            self.parameters['W'+str(layer_idx)] = np.random.randn(layer_output,layer_input)
            self.parameters['B'+str(layer_idx)] = np.random.randn(layer_output,1)

    def forward_propagation(self, inputs):
        inputs = np.array(inputs, ndmin=2).T
        out_curr = inputs
        memory = {}
        for idx, layer in enumerate(self.architecture):
            layer_idx = idx+1
            w_curr = self.parameters["W"+str(layer_idx)]
            b_curr = self.parameters["B"+str(layer_idx)]
            out_curr = forward_single_layer(out_prev=out_curr,w_curr=w_curr,b_curr=b_curr)
            memory[idx] = out_curr
        return out_curr, memory


    def train_iteration(self, inputs, target, learn_rate):
        target = np.array(target, ndmin=2).T
        output, memory = self.forward_propagation(inputs)
        inputs = np.array(inputs, ndmin=2).T
        loss = 0
        for idx, layer in reversed(list(enumerate(self.architecture))):
            layer_idx = idx+1
            if layer_idx == len(self.architecture):
                loss += target - output
                adjust_w = 1/layer["inputs"]*learn_rate*(loss*d_activation_func(memory[idx])).dot(memory[idx-1].T)
                adjust_b = 1/layer["inputs"]*learn_rate*loss*d_activation_func(memory[idx])
                self.parameters["W"+str(layer_idx)] += (adjust_w/learn_rate - adjust_b.dot(np.ones((1,adjust_w.shape[1]))))*learn_rate
                self.parameters["B"+str(layer_idx)] += adjust_b
            elif layer_idx == 1:
                loss = self.parameters["W"+str(layer_idx+1)].T.dot(loss)
                adjust_w = 1/layer["inputs"]*learn_rate*(loss*d_activation_func(memory[idx])).dot(inputs.T)
                adjust_b = 1/layer["inputs"]*learn_rate*loss*d_activation_func(memory[idx])
                self.parameters["W"+str(layer_idx)] += (adjust_w/learn_rate - adjust_b.dot(np.ones((1,adjust_w.shape[1]))))*learn_rate
                self.parameters["B"+str(layer_idx)] += adjust_b
            else:
                loss = self.parameters["W"+str(layer_idx+1)].T.dot(loss)
                adjust_w = 1/layer["inputs"]*learn_rate*(loss*d_activation_func(memory[idx])).dot(memory[idx-1].T)
                adjust_b = 1/layer["inputs"]*learn_rate*loss*d_activation_func(memory[idx])
                self.parameters["W"+str(layer_idx)] += (adjust_w/learn_rate - adjust_b.dot(np.ones((1,adjust_w.shape[1]))))*learn_rate
                self.parameters["B"+str(layer_idx)] += adjust_b

    def train(self, inputs, targets):
        for i in range(0,imagenumb):
            inputdata = inputs[i,:]
            target = targets[i,:]
            self.train_iteration(inputdata, target, 0.1)
            if i/imagenumb*100 % 10 == 0:
                print('Training state: ', i/imagenumb*100, ' %')

NN = NeuralNetwork(NN_architecture)

### Train Neural Network


In [67]:
NN.train(train_data,train_target)

count = 0
number = np.empty(10)
for i in range(len(train_tar)):
    result, mem = NN.forward_propagation(train_data[i,:])
    result = np.argmax(result)
    if result == train_tar[i]:
        count += 1
        number[result] += 1

print('Performance: ', count/len(train_tar)*100, '%')

Training state:  0.0  %
Training state:  10.0  %
Training state:  20.0  %
Training state:  30.0  %
Training state:  40.0  %
Training state:  50.0  %
Training state:  60.0  %
Training state:  70.0  %
Training state:  80.0  %
Training state:  90.0  %
Performance:  90.0047619047619 %


### Test Neural Network and Submission

In [68]:
test = pd.read_csv('test.csv')
test_data = np.array(test)
test_data = 0.99*test_data/255 + 0.1

prediction = np.empty(test_data.shape[0])

for i in range(test_data.shape[0]):
    pred, mem = NN.forward_propagation(test_data[i,:])
    prediction[i] = np.argmax(pred)

In [69]:
ID = np.arange(1,test_data.shape[0]+1)

In [70]:
ID = np.array(ID, ndmin=2, dtype=int).T
prediction = np.array(prediction, ndmin=2, dtype=int).T

sub = np.hstack((ID,prediction))
sub = pd.DataFrame(sub, columns=['ImageId', 'Label'])
sub.to_csv('sub.csv', index=False)
